<a href="https://colab.research.google.com/github/josha106/Dacon-First-Medical-AI-Contest/blob/main/%EC%A0%9C1%ED%9A%8C_Medical_AI_(MAI)_%EA%B2%BD%EC%A7%84%EB%8C%80%ED%9A%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import pandas as pd
import numpy as np
import os
import re
import glob
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

from tqdm import tqdm

import warnings
warnings.filterwarnings(action='ignore')

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.18 (you have 1.4.15). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [ ]:
CFG = {
    'IMG_SIZE':224,
    'EPOCHS':20,
    'LEARNING_RATE':3e-4,
    'BATCH_SIZE':32,
    'SEED':41
}

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def transform_path(df):
  updated_paths = []
  df = df

  for path in df['path'].values:
      new_path = path.replace('./train/', '/content/drive/MyDrive/open/train/')
      updated_paths.append(new_path)

  return updated_paths

def transform_test_path(df):
  updated_paths = []
  df = df

  for path in df['path'].values:
      new_path = path.replace('./test/', '/content/drive/MyDrive/open/test/')
      updated_paths.append(new_path)

  return updated_paths

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/open/train.csv')
updated_paths = transform_path(df)
df['path'] = updated_paths

# train(80%) / valid(20%)
train_len = int(len(df) * 0.8)
train_df = df.iloc[:train_len]
val_df = df.iloc[train_len:]

# 2차원 벡터화
train_label_vec = train_df.iloc[:,2:].values.astype(np.float32)
val_label_vec = val_df.iloc[:,2:].values.astype(np.float32)

CFG['label_size'] = train_label_vec.shape[1]

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms

    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        image = cv2.imread(img_path)

        # image A.compose로 규격 맞추기
        if self.transforms is not None:
            image = self.transforms(image=image)['image']

        if self.label_list is not None:
            label = self.label_list[index]
            return image, label
        else:
            return image

    def __len__(self):
        return len(self.img_path_list)

In [ ]:
A_Norm_toTensor = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])
augmentedData = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            A.OneOf([
                                    A.HorizontalFlip(p=0.5),
                                    A.RandomRotate90(p=0.5),
                                    A.VerticalFlip(p=0.5)
                                    ], p=1),
                            A.OneOf([
                                    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=0, p=1.0)
                                    ], p=1),
                            ToTensorV2()
                            ])

In [ ]:
# 1. 기존의 이미지 -> tensor
train_dataset = CustomDataset(train_df['path'].values, train_label_vec, A_Norm_toTensor)
# 2. Data Augmentation
train_augmented_1 = CustomDataset(train_df['path'].values, train_label_vec, augmentedData)
train_augmented_2 = CustomDataset(train_df['path'].values, train_label_vec, augmentedData)
train_concat = torch.utils.data.ConcatDataset([train_dataset, train_augmented_1, train_augmented_2])
# 3. 1+2(concatenate) -> DataLoader
train_loader = DataLoader(train_concat, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

In [ ]:
val_dataset = CustomDataset(val_df['path'].values, val_label_vec, A_Norm_toTensor)
val_augmented_1 = CustomDataset(train_df['path'].values, train_label_vec, augmentedData)
val_augmented_2 = CustomDataset(train_df['path'].values, train_label_vec, augmentedData)
val_concat = torch.utils.data.ConcatDataset([val_dataset, val_augmented_1, val_augmented_2])
val_loader = DataLoader(train_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

In [ ]:
# Feature Extraction(resnext101_32x8d) + FFNN(2layer)
class BaseModel(nn.Module):
    def __init__(self, gene_size=CFG['label_size'], fc_dropout_prob=0.3):
        super(BaseModel, self).__init__()

        # Pretrained ResNet (여기서는 ResNext101)
        self.backbone = models.resnext101_32x8d(pretrained=True)

        # 추가적인 fully connected 레이어 및 dropout
        self.fc1 = nn.Linear(1000, 512)  # 추가 레이어
        self.fc_dropout = nn.Dropout(p=fc_dropout_prob)
        self.fc2 = nn.Linear(512, gene_size)  # 최종 회귀층

    def forward(self, x):
        # ResNet backbone을 통해 feature 추출
        x = self.backbone(x)
        x = self.fc_dropout(x)

        # 추가 FC 레이어 적용
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc_dropout(x)

        # 최종 출력 레이어
        x = self.fc2(x)
        return x

In [ ]:
def train(model, optimizer, train_loader, val_loader, scheduler, device, epochs=CFG['EPOCHS']):
    model.to(device)
    criterion = nn.MSELoss().to(device)

    best_loss = 99999999
    best_model = None

    for epoch in range(1, epochs+1):
        model.train()
        train_loss = []
        for imgs, labels in tqdm(iter(train_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            output = model(imgs)
            loss = criterion(output, labels)

            loss.backward()
            optimizer.step()

            train_loss.append(loss.item())

        _val_loss = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}]')

        if scheduler is not None:
            scheduler.step(_val_loss)

        if best_loss > _val_loss:
            best_loss = _val_loss
            best_model = model

    return best_model

In [ ]:
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []

    with torch.no_grad():
        for imgs, labels in tqdm(iter(val_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)

            pred = model(imgs)

            loss = criterion(pred, labels)

            val_loss.append(loss.item())

        _val_loss = np.mean(val_loss)

    return _val_loss

In [ ]:
model = BaseModel()
# Load sota model
model.load_state_dict(torch.load('/content/drive/MyDrive/resnext_2drop.pth'))
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, threshold_mode='abs', min_lr=1e-8, verbose=True)

infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

Downloading: "https://download.pytorch.org/models/resnext101_32x8d-8ba56ff5.pth" to /root/.cache/torch/hub/checkpoints/resnext101_32x8d-8ba56ff5.pth
100%|██████████| 340M/340M [00:02<00:00, 131MB/s]
100%|██████████| 175/175 [01:53<00:00,  1.54it/s]


Epoch [1], Train Loss : [0.04645] Val Loss : [0.04605]


100%|██████████| 175/175 [01:51<00:00,  1.58it/s]


Epoch [2], Train Loss : [0.04655] Val Loss : [0.04548]


100%|██████████| 175/175 [01:49<00:00,  1.60it/s]


...

Epoch [18], Train Loss : [0.04535] Val Loss : [0.04483]


100%|██████████| 175/175 [01:49<00:00,  1.59it/s]


Epoch [19], Train Loss : [0.04533] Val Loss : [0.04515]


100%|██████████| 175/175 [01:50<00:00,  1.58it/s]

Epoch [20], Train Loss : [0.04530] Val Loss : [0.04497]


In [ ]:
test = pd.read_csv('/content/drive/MyDrive/open/test.csv')
updated_paths = transform_test_path(test)

test_dataset = CustomDataset(updated_paths, None, A_Norm_toTensor)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

def inference(model, test_loader, device):
    model.eval()
    preds = []
    with torch.no_grad():
        for imgs in tqdm(test_loader):
            imgs = imgs.to(device).float()
            pred = model(imgs)

            preds.append(pred.detach().cpu())

    preds = torch.cat(preds).numpy()

    return preds

preds = inference(infer_model, test_loader, device)

100%|██████████| 72/72 [00:41<00:00,  1.75it/s]


In [ ]:
test_dataset_augmented = CustomDataset(updated_paths, None, augmentedData)
test_loader_augmented = DataLoader(test_dataset_augmented, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)


preds_augmented = inference(infer_model, test_loader_augmented, device)

100%|██████████| 72/72 [00:54<00:00,  1.33it/s]


In [ ]:
test_dataset_augmented_1 = CustomDataset(updated_paths, None, augmentedData)
test_loader_augmented_1 = DataLoader(test_dataset_augmented, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)


preds_augmented_1 = inference(infer_model, test_loader_augmented, device)

100%|██████████| 72/72 [00:56<00:00,  1.28it/s]


In [ ]:
# test 이미지에 대한 augmentation(한 이미지를 여러 방향에서 바라본 후), 3개의 이미지에 대한 평균 값 제출
average_preds = (preds + preds_augmented + preds_augmented_1) / 3

In [ ]:
submit = pd.read_csv('/content/drive/MyDrive/open/sample_submission.csv')
submit.iloc[:, 1:] = np.array(average_preds).astype(np.float32)
submit.to_csv('./baseline_submit.csv', index=False)